In [75]:
import requests
import pandas as pd
import numpy as np
import pulp

# Use API to Grab Current Team

In [53]:
history = pd.read_pickle('/Users/andrewpeters/GitHub/fpl/data/interim/df.pkl')
current_gw = int(history[history['finished'] == True].sort_values('kickoff_time')['gw'].tail(1))

In [54]:
url = f'https://fantasy.premierleague.com/api/entry/5171826/event/{current_gw}/picks/'
r = requests.get(url)
json = r.json()

In [87]:
current_team = pd.DataFrame(json['picks'])
money_remaining = json['entry_history']['bank']

In [66]:
current_team = current_team.merge(history[['element', 'player']], how='left').drop_duplicates().reset_index(drop=True)

# Solve Knapsack Problem

In [109]:
df = pd.read_pickle('/Users/andrewpeters/GitHub/fpl/data/processed/grouped_predictions_with_cost.pkl')

In [110]:
df.gw = df.gw.astype('int64')
data = df[df.gw.isin(df.gw.unique()[:5])] #filter for next 5 gws only
data = data.groupby(['player', 'position', 'team', 'now_cost', 'element'])['predicted_points'].sum().reset_index()

expected_scores = data.predicted_points.values
num_players = len(data)
prices = data.now_cost.values / 10
positions = data.position.values
teams = data.team.values
names = data.player.values
budget_now = money_remaining/10

In [118]:
from transfer_optimizer import TransferOptimiser

In [113]:
opt = TransferOptimiser(expected_scores, buy_prices, sell_prices, positions, teams)

In [114]:
current_squad_indices = data[data.element.isin(current_team.element)].index #look up indices for current team

In [115]:
transfer_in_decisions, transfer_out_decisions, starters, sub_decisions, captain_decisions = opt.solve(current_squad_indices, budget_now, sub_factor=0.2)

/Users/andrewpeters/anaconda3/lib/python3.8/site-packages/pulp/pulp.py:1198: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Solver status: 1


In [116]:
for i in range(num_players):
    if transfer_in_decisions[i].value() == 1:
        print("Transferred in: {} {} {:.2f}".format(names[i], buy_prices[i], expected_scores[i]))
    if transfer_out_decisions[i].value() == 1:
        print("Transferred out: {} {} {:.2f}".format(names[i], sell_prices[i], expected_scores[i]))

Transferred out: Alex_McCarthy 4.5 8.80
Transferred in: Alexis_Mac Allister 5.3 90.00
Transferred in: Anwar_El Ghazi 5.3 87.00
Transferred in: Ayoze_Pérez 6.0 90.00
Transferred out: Ben_Mee 5.1 16.03
Transferred out: Bruno Miguel_Borges Fernandes 11.6 11.55
Transferred in: Danny_Ward 4.3 45.00
Transferred out: Dominic_Calvert-Lewin 7.6 25.23
Transferred out: Emiliano_Martínez 5.3 30.80
Transferred out: Ilkay_Gündogan 6.1 31.23
Transferred out: Jack_Grealish 7.7 30.80
Transferred in: Jan_Bednarek 4.9 45.00
Transferred in: Jannik_Vestergaard 4.7 45.00
Transferred out: João Pedro Cavaco_Cancelo 6.1 31.23
Transferred in: Kelechi_Iheanacho 5.6 90.00
Transferred in: Matej_Vydra 4.8 72.00
Transferred in: Max_Kilman 4.0 72.00
Transferred out: Mohamed_Salah 12.6 6.24
Transferred out: Patrick_Bamford 6.9 7.80
Transferred out: Pedro_Lomba Neto 5.7 20.53
Transferred in: Ross_Barkley 5.9 108.00
Transferred out: Rúben Santos_Gato Alves Dias 6.1 15.43
Transferred in: Steven_Alzate 4.2 90.00
Transferr

In [103]:
player_indices = []
print()
print("First Team:")
for i in range(len(starters)):
    if starters[i].value() == 1:
        print("{}{}".format(names[i], "*" if captain_decisions[i].value() == 1 else ""))
        player_indices.append(i)
print()
print("Subs:")
for i in range(len(sub_decisions)):
    if sub_decisions[i].value() == 1:
        print(names[i])
        player_indices.append(i)


First Team:
Ahmed_El Mohamady
Alexis_Mac Allister
Ayoze_Pérez
Jed_Steer
Jimmy_Dunne
Kelechi_Iheanacho
Matej_Vydra
Max_Kilman
Ross_Barkley*
Steven_Alzate
Willian José_Da Silva

Subs:
Ilkay_Gündogan
João Pedro Cavaco_Cancelo
Thomas_McGill
Vladimir_Coufal
